## Obtain process impacts by combining process outputs with impact factors

Inputs: - Process Excel file downloaded from OpenLCA
        - Impact factor conversion xml files downloaded from OpenLCA

Output: - Csv file of impact for each gas for all processes in input file

In [1]:
# Import packages
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
pd.options.mode.chained_assignment = None

In [2]:
# Variable definition
data_path = "C:/Users/lukec/OneDrive - University of Cambridge/PhD/Data/EcoInvent/"
lcaDataPath = data_path+"full_LCI/Consequential_v3_9_csv.csv"
impactMethods_20 = data_path+"impact_methods/EI_3_8_IPCC2013_GWP20a.xml"
impactMethods_100 = data_path+ "impact_methods/EI_3_8_IPCC2013_GWP100a.xml"

output_path = '../data/extracted/'

In [3]:
# Define functions
def to_listlist(inlist: list):
    """Ensures element is a list of lists even if single inner list"""
    return [inlist] if type(inlist[0]) is not list else inlist

def read_xml_attributes(filepath:str,branches:list,attributes:list,df=False):
    """This function creates a dataframe of attributes within an xml file.\n
    Inputs:\n
    filepath - path to xml file\n
    branches - list of successive branch choices\n
    attributes - attributes to be read from chosen branch\n
    Outputs:\n
    df - pandas dataframe of attributes for each end branch\n
    Requirements: pandas as pd, xml.etree.ElementTree as ET"""
    if df is False: df = pd.DataFrame([],columns=[i for sublist in to_listlist(attributes) for i in sublist])
    tree_loc = ET.parse(filepath). getroot()
    branches, attributes = to_listlist(branches), to_listlist(attributes)
    attr_values = dict()
    for pathnum, path in enumerate(branches):
        for branch in path:
            for num,val in enumerate([branch in i.tag for i in tree_loc]):
                if val:
                    if branch is path[-1]:
                        attr_values.update(dict(zip(attributes[pathnum],[tree_loc[num].attrib[i] if i else tree_loc[num].text for i in attributes[pathnum]])))
                        if path is branches[-1]:
                            df = pd.concat([df, pd.DataFrame(np.array([[i] for i in attr_values.values()]).transpose(),columns=attr_values.keys())],ignore_index=True)
                    else:
                        tree_loc=tree_loc[num]
                        break
    return df

In [4]:
# Fetch and display impact factors
impacts_20, impacts_100 = [read_xml_attributes(input_file,['dataset','flowData','exchange'],['name','category','subCategory','meanValue']).rename(columns={'meanValue':name}) for input_file, name in zip([impactMethods_20, impactMethods_100], ['CO2e_20a', 'CO2e_100a'])]

impacts = impacts_20.merge(impacts_100, on=list(impacts_20.columns[:-1])).sort_values('name').reset_index(drop=True)

impacts.to_csv(output_path+'EI_3_8_IPCC2013_CO2e.csv')
impacts

,name,category,subCategory,CO2e_20a,CO2e_100a
0,"Carbon dioxide, fossil",Emission to air,low population density,1.0,1.0
1,"Carbon dioxide, fossil",Emission to air,unspecified,1.0,1.0
2,"Carbon dioxide, fossil",Emission to air,"low population density, long-term",1.0,1.0
3,"Carbon dioxide, fossil",Emission to air,lower stratosphere + upper troposphere,1.0,1.0
4,"Carbon dioxide, fossil",Emission to air,high population density,1.0,1.0
...,...,...,...,...,...
206,Sulfur hexafluoride,Emission to air,high population density,17499.90776451,23506.81999316
207,Sulfur hexafluoride,Emission to air,unspecified,17499.90776451,23506.81999316
208,Sulfur hexafluoride,Emission to air,"low population density, long-term",17499.90776451,23506.81999316
209,Sulfur hexafluoride,Emission to air,low population density,17499.90776451,23506.81999316


In [5]:
# Calculate emissions from LCA data inventories
# Load impact factors
impacts = pd.read_csv(output_path+'EI_3_8_IPCC2013_CO2e.csv', index_col=0)

# Load process outputs
processes_raw = pd.read_csv(lcaDataPath, header=[0,1,2], low_memory=False)

processes = processes_raw.rename(columns=dict(zip(processes_raw.iloc[0].reset_index()['level_0'][:5], processes_raw.iloc[0].values[:5])))[1:]
processes = processes.set_index(list(processes.columns[:6])).astype(float)

In [6]:
processes

1,2-Dichlorobenzene  \
                                                                                                                                                                                                                                                                                                                                  air   
                                                                                                                                                                                                                                                                                                            urban air close to ground   
(Activity UUID_Product UUID, Unnamed: 0_level_1... (Activity Name, Unnamed: 1_level_1, Unnamed: 1_... (Geography, Unnamed: 2_level_1, Unnamed: 2_leve... (Reference Product Name, Unnamed: 3_level_1, Un... (Reference Product Unit, Unnamed: 4_level_1, Un... (Exchange Name, Compartment, Subcompartment)                             
c527b1db-ee62-5322-b806-679f4a317e3a_807906d0-f... 1,1-difluoroethane production, HFC-152a            RoW                                                1,1-difluoroethane, HFC-152a                       kg                                                 1                                                        -2.974310e-10   
4be0d9cf-b5f5-5074-b618-3d9af1b9e239_807906d0-f... 1,1-difluoroethane production, HFC-152a            US                                                 1,1-difluoroethane, HFC-152a                       kg                                                 1                                                        -3.830500e-10   
8847e7d2-bf94-51a3-afdc-57835fc0e2ce_41bae23f-2... 1,1-dimethylcyclopentane to generic market for ... GLO                                                1,1-dimethylcyclopentane                           kg                                                 1                                                         7.309770e-11   
a6564925-eec5-5594-9b01-77ec9251561a_95aedd09-3... 1-methoxy-2-propanol production                    GLO                                                1-methoxy-2-propanol                               kg                                                 1                                                         6.362820e-12   
314fd77e-fb24-5104-a130-cc6a45583835_e24e6570-e... 1-methylcyclopropene production                    GLO                                                1-methylcyclopropene                               kg                                                 1                                                         3.094560e-12   
...                                                                                                                                                                                                                                                                                                                               ...   
dc5e84bc-51e8-5681-854d-2a1c4440ec59_08663c56-b... zirconium oxide to generic market for electroly... GLO                                                electrolyte, for solid oxide fuel cell             kg                                                 1                                                         1.805930e-10   
a41d149b-f51d-52e4-b34a-39a133344781_11985fce-6... zirconium sponge production, from zirconium tet... FR                                                 zirconium sponge, nuclear-grade                    kg                                                 1                                                        -1.225910e-09   
0606da77-7e3a-5566-a49f-0733ec65e9ea_11985fce-6... zirconium sponge production, from zirconium tet... RoW                                                zirconium sponge, nuclear-grade                    kg                                                 1                                                         3.852520e-10   
f8cc5fc3-0212-5318-9746-f0cfd

In [126]:
elements = processes_raw.transpose().reset_index()[['level_0', 'level_1', 'level_2']]
conversions = elements.merge(impacts, left_on=['level_0', 'level_2'], right_on=['name', 'subCategory'], how='inner').set_index(['name', 'level_1', 'subCategory'])

In [185]:
## Calculate CO2e emissions

def calc_emissions(processes:pd.DataFrame, conversions:pd.DataFrame, conversion_col:str='CO2e_100a') -> np.ndarray:
    """Function calculates the CO2e emissions given process outputs and conversion factors"""
    emitting_cols = processes[conversions.index]
    for column in conversions.index:
        emitting_cols[column] = emitting_cols[column].values*conversions.loc[column][conversion_col]

    return emitting_cols.sum(axis=1).values

output_df = processes[[]].reset_index()
output_df.columns = output_df.columns.droplevel([1, 2])
output_df.rename(columns={'Exchange Name': 'Reference Value'}, inplace=True)
output_df['CO2e_20a'] = calc_emissions(processes, conversions, 'CO2e_20a')
output_df['CO2e_100a'] = calc_emissions(processes, conversions, 'CO2e_100a')

In [187]:
# Save output to file
output_df.to_csv(output_path+'conversionFactors_from_ecoinvent_IPCC2013.csv', index=False)